In [42]:
# Imports
import requests        # For fetching news from Finnhub
import os              # For reading environment variables
from dotenv import load_dotenv   # For loading API keys from .env
import re              # For text cleaning
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer  # For sentiment analysis

# Load API key from .env file
load_dotenv()
finnhub_key = os.getenv("FINNHUB_API_KEY")

# Check if key is loaded
if not finnhub_key:
    raise ValueError("⚠️ Please add your FINNHUB_API_KEY to .env file")

print("Finnhub Key loaded:", finnhub_key[:5] + "...")  # Show first 5 chars only

Finnhub Key loaded: d5k5v...


In [43]:
# Function to clean text headlines
def clean_text(text):
    if not text:
        return ""
    
    # Remove leading/trailing spaces
    text = text.strip()
    
    # Remove unwanted symbols (keep letters, numbers, and basic punctuation)
    text = re.sub(r"[^\w\s.,?!]", "", text)
    
    # Replace multiple spaces with a single space
    text = re.sub(r"\s+", " ", text)
    
    return text

# Initialize VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to classify sentiment of text
def get_sentiment_vader(text):
    scores = analyzer.polarity_scores(text)
    compound = scores['compound']  # Overall sentiment score
    
    if compound >= 0.05:
        return "Bullish"
    elif compound <= -0.05:
        return "Bearish"
    else:
        return "Neutral"

In [44]:
# Function to fetch top 5 news headlines from Finnhub
def fetch_news(ticker, from_date="2026-01-10", to_date="2026-01-14"):
    url = f"https://finnhub.io/api/v1/company-news?symbol={ticker}&from={from_date}&to={to_date}&token={finnhub_key}"
    
    try:
        data = requests.get(url).json()  # Fetch data from API
    except Exception as e:
        print("Error fetching news:", e)
        return []
    
    headlines = []
    for item in data[:5]:  # Get only top 5 headlines
        title = clean_text(item.get("headline", ""))  # Clean the headline
        link = item.get("url", "")                   # Get URL of news
        headlines.append({"title": title, "link": link})
    
    if not headlines:
        print("⚠️ No headlines found. Check ticker or date range!")
    
    return headlines

In [48]:
# Fetch news for a stock ticker
ticker = "AAPL"
news = fetch_news(ticker)

# Print each headline with link and sentiment
for i, item in enumerate(news, 1):
    title = item['title']
    sentiment = get_sentiment_vader(title)  # Get sentiment using VADER
    
    print(f"{i}. {title}")
    print(f"   Link: {item['link']}")
    print(f"   Sentiment: {sentiment}\n")

1. TSMC likely to post fourthquarter profit leap driven by AI boom
   Link: https://finnhub.io/api/news?id=b9fa2f285e852b27a13beb4b549ac91f9914943d44cbcf4239bae46908db7300
   Sentiment: Bullish

2. Is Apples AI partnership with Google a major loss for OpenAI?
   Link: https://finnhub.io/api/news?id=9645477a144fc94aca9c12c5b46b4c190f88604329d4d0dfac9c2a9f10a4bf1e
   Sentiment: Bearish

3. Elon Musk Slams Google, Apple Deal Unreasonable Concentration Of Power
   Link: https://finnhub.io/api/news?id=f0b88307180aff7c9fe5f1956a063f533cac24c5ff79856ce6c9d5e7b5ce3e05
   Sentiment: Neutral

4. Intel Stock Hits Highest Level in Nearly 2 Years. What Has the Market Excited.
   Link: https://finnhub.io/api/news?id=63bc261e5c8c6307717115fadd9be1b64db38698fbe7f4a81b0fa7aebc0c5433
   Sentiment: Bullish

5. Google vs. Apple Which Magnificent 7 Stock is a Better Buy Right Now?
   Link: https://finnhub.io/api/news?id=55ae754bc07fab89c7dcd2ef1f877902a498fbf64ba13f302f21c211b2bcdca3
   Sentiment: Bullish
